# old 方法: YOLOv4

In [ ]:
import sys
sys.path.append("..")

import pandas as pd
from models import Yolov4
from tensorflow import keras
from config import yolo_config
from iou import get_iou, calculate_ciou
from utils import DataGenerator, read_annotation_lines

import numpy as np
from glob import glob
from tqdm.notebook import tqdm

import seaborn as sns
from turtle import color
from matplotlib import style
import matplotlib.pylab as pl
import matplotlib.pyplot as plt

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

import math
import cv2 as cv
import xml.etree.ElementTree as ET

import os
# 使用第二張 GPU 卡
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# 載入模型

In [ ]:
# yolo_config['score_threshold']=0.7
# print(yolo_config)

In [ ]:

class_name_path = '../class_names/classes.txt'
model = Yolov4(
               weight_path    = None, 
               config         = yolo_config,
               class_name_path= class_name_path, 
               )
model.load_weights(path="../model/202303/20230331_sopbox_0.weights")

# 使用 stage1 分類預測結果的 "真值"

In [ ]:
stage1_pred_result = pd.read_pickle("../../classification/result/stage1_pred_result_test_20230323.pickle")
# stage1_pred_result = pd.read_pickle("../../classification/result/stage1_pred_result_train.pickle")   # 這邊換stage1結果!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# stage1_pred_result = pd.read_pickle("../../classification/result/stage1_pred_result_test.pickle")   # 這邊換stage1結果!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# stage1_pred_result = pd.read_pickle("../../classification/result/stage1_pred_result_validation.pickle")   # 這邊換stage1結果!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# stage1_pred_result = pd.read_pickle("../../classification/result/stage1_pred_result_test_nosmooth.pickle")   # 這邊換stage1結果!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
stage1_pred_result = stage1_pred_result.drop(columns=['stage1_test_pred'])
stage1_pred_result

# 每一個test image "True" bounding box

In [ ]:
true_box_PATH = '../test_dataset/txt/20230330_anno_test_lines_0.txt'
# true_box_PATH = '../test_dataset/txt/20221102_anno_test_lines_0.txt'       # 這邊要修改train/test 的路徑!!!!!!!!
# true_box_PATH = '../dataset/txt/20221102_anno_train_lines_0.txt'
# true_box_PATH = '../dataset/txt/20221102_anno_val_lines_0.txt'


true_bboxes_names = []
true_bboxes_list  = []
with open(true_box_PATH) as f:
    for line in f.readlines():
        true_bboxes_list.append(tuple([int(i) for i in line.split(' ')[1].split(',')[:4]]))
        true_bboxes_names.append(line.split(' ')[0])
        

In [ ]:
true_bboxes = pd.DataFrame({'test_name':true_bboxes_names, 
                            'true_bbox':true_bboxes_list
                          })
true_bboxes

In [ ]:
result_table = stage1_pred_result.merge(true_bboxes, on=['test_name'], how='left')
result_table = result_table.fillna('NO')
result_table

# 每一個test image "Pred" bounding box

In [ ]:
# PATH = '../../classification/dataset_smooth_186video_20230323/test/0/'
PATH = '../../classification/dataset_186video_20230323/test/0/'
# PATH = '../../classification/dataset_22video_20221031/test/0/'
# PATH = '../../classification/dataset_smooth_22video_20221031/test/0/'
# PATH = '../../classification/dataset_smooth_22video_20221031/test/0/'
# PATH = '../dataset/img/20221024_train_vali_img_sopbox_0/'                        # 這邊要修改train/test 的路徑!!!!!!!!

pred_bboxes_names  = []  # all image names
pred_bboxes_scores = []  # all bboxes scores
pred_bboxes_list   = []  # all bboxes
for test_img_name in result_table[result_table['test_true']==0]['test_name']:  # glob(os.path.join(PATH, '*.png')):
    test_img_path = PATH + test_img_name
    output_img, detections = model.predict(img_path=test_img_path, 
                                            random_color=False, # 框顏色
                                            plot_img=False, # 顯示出框選圖片
                                            show_text=True, # 框上面加上類別文字
                                            box_num=1,      # 顯示框的數量
                                            return_output=True)
    

    pred_box_num = len(detections['score'])  
    if pred_box_num > 0:       # 有預測出box
        score = detections[0:1]['score'].to_numpy()[0]
        pred_bboxes_names.append(test_img_name)
        pred_bboxes_scores.append(score)
        pred_bboxes_list.append(tuple(detections[0:1][['x1','y1','x2','y2']].to_numpy()[0]))

    else:
        pred_bboxes_names.append(test_img_name)
        pred_bboxes_scores.append(0.0)
        pred_bboxes_list.append('NO')
        print('沒有預測出來有box(label)')

PATH = '../../classification/dataset_186video_20230323/test/1/'  #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  要記得換
# PATH = '../../classification/dataset_smooth_186video_20230323/test/1/'  #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  要記得換
# PATH = '../../classification/dataset_22video_20221031/test/1/'  #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  要記得換
# PATH = '../../classification/dataset_smooth_22video_20221031/test/1/'
for test_img_name in result_table[result_table['test_true']==1]['test_name']:  # glob(os.path.join(PATH, '*.png')):
    test_img_path = PATH + test_img_name
    output_img, detections = model.predict(img_path=test_img_path, 
                                            random_color=False, # 框顏色
                                            plot_img=False, # 顯示出框選圖片
                                            show_text=True, # 框上面加上類別文字
                                            box_num=1,      # 顯示框的數量
                                            return_output=True)
    
    pred_box_num = len(detections['score'])                    
    if pred_box_num > 0:       # 有預測出box
        score = detections[0:1]['score'].to_numpy()[0]
        pred_bboxes_names.append(test_img_name)
        pred_bboxes_scores.append(score)
        pred_bboxes_list.append(tuple(detections[0:1][['x1','y1','x2','y2']].to_numpy()[0]))

    else:
        pred_bboxes_names.append(test_img_name)
        pred_bboxes_scores.append(0.0)
        pred_bboxes_list.append('NO')
        print('沒有預測出來有box(label)')

In [ ]:
pred_bboxes = pd.DataFrame({'test_name':pred_bboxes_names, 
                            'pred_bbox':pred_bboxes_list,
                            'score':pred_bboxes_scores
               })
pred_bboxes

# Left join table

In [ ]:
result_table          = result_table.merge(pred_bboxes, on=['test_name'], how='left')
result_table['score'] = result_table['score'].fillna(0)  # 分數補0
result_table          = result_table.fillna('NO')       # bbox補'NO'
result_table

# 計算iou
參數:

iou_threshold = 0.7    "iou大於0.7才是有框選到"

In [ ]:
def valid(TP, FN, FP, TN):
    
    precision = 0 if (TP+FP)==0 else TP/(TP+FP)
    recall    = 0 if (TP+FN)==0 else TP/(TP+FN)
    f1        = 0 if (precision+recall)==0 else 2*precision*recall/(precision+recall)

    print('precision={:.2f}, recall={:.2f}, f1={:.2f}'.format(precision, recall, f1))
    print(TP,"|",FN,"\n",
          FP,"|",TN)

    return precision, recall, f1

In [ ]:
def plt_precision_recall_curve_1(y_true, y_scores):
    precision, recall, thresholds = precision_recall_curve(y_true, y_scores)
    average_precision             = average_precision_score(y_true, y_scores)
    # 用套件畫
    # disp = PrecisionRecallDisplay(precision=precision, recall=recall, average_precision=average_precision)
    # disp.plot()
    # plt.ylim([0.0, 1.05])
    # plt.xlim([0.0, 1.0])
    # plt.show()

    # 自己畫
    plt.plot(recall, precision, 'b-')   
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])  
    plt.xlabel('recall')
    plt.ylabel('precision')
    plt.title('precision_recall_curve')
    plt.legend(['AP={}'.format(round(average_precision,2))])
    plt.show()

    # print('precision = ', precision)
    # print('recall = ', recall)
    # print('thresholds = ',thresholds)
    print('AP={:.2f}'.format(average_precision))

In [ ]:
def plt_heatmap(con_ma):
    '''
    :input: con_ma:[[TP,FN],[FP,TN]]
    '''
    sns.set(font_scale=1.5)
    f,ax=plt.subplots()
    sns.heatmap(con_ma, annot=True, ax=ax, fmt='d', cmap='YlGnBu', vmax=200, vmin=0) # 畫熱力圖
    ax.set_title('confusion matrix') # 標題
    ax.set_xlabel('predict')         # x軸
    ax.set_ylabel('true')            # y軸
    ax.xaxis.set_ticklabels(['Papilla', 'No Papilla'])
    ax.yaxis.set_ticklabels(['Papilla', 'No Papilla'])
    plt.show()

# old測試集結果 ciou_threshold=0.5:0.05:0.95

In [ ]:
row, column = result_table.shape
result_table

In [ ]:
def plt_precision_recall_curve_2(iou_thresholds, y_trues, y_scores):
    n      = len(y_trues)
    colors = pl.cm.jet(np.linspace(0,1,n))
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])  
    plt.xlabel('recall')
    plt.ylabel('precision')
    plt.title('precision_recall_curve')

    total_average_precision = 0
    for i in range(n):
        iou_threshold = round(iou_thresholds[i],4)
        y_true        = y_trues[i]
        y_score       = y_scores[i]
        average_precision = round(average_precision_score(y_true, y_score), 4)
        if average_precision<=0 or math.isnan(average_precision):
            total_average_precision += 0
            plt.plot([0], [0], color=colors[i], label = "IoU={} => AP={}".format(iou_threshold, 0))
        else:
            total_average_precision += average_precision
            precision, recall, thresholds = precision_recall_curve(y_true, y_score)
            # 自己畫
            plt.plot(recall, precision, color=colors[i], label = "IoU={} => AP={}".format(iou_threshold, average_precision))   
        
        # print('precision = ', precision)
        # print('recall = ', recall)
        # print('thresholds = ',thresholds)
        # print('AP={:.2f}'.format(average_precision))

    averaged_AP = round(total_average_precision/n, 4)
    print("Averaged_AP= ", averaged_AP)
    plt.legend(loc='upper left',bbox_to_anchor=(0.0, -0.3))
    plt.show()

In [ ]:
iou_thresholds = np.arange(0.5, 1.0, 0.05)      

precisions, recalls, f1s = [], [], []
y_trues  = []
y_scores = []
for iou_threshold in iou_thresholds:
    y_true   = []                # 0:無壺腹乳頭 1:有壺腹乳頭
    y_score  = []                # confidence分數
    for r in range(row):         # row: 列數
        test_img_name = result_table.iloc[r:r+1]['test_name'][r]
        true_bbox     = result_table.iloc[r:r+1]['true_bbox'][r]
        pred_bbox     = result_table.iloc[r:r+1]['pred_bbox'][r]
        score         = result_table.iloc[r:r+1]['score'][r]

        if true_bbox != 'NO' and pred_bbox != 'NO':
            iou = get_iou(true_bbox, pred_bbox)           # 找出真實box和預測box的iou
            # ciou = calculate_ciou(true_bbox, pred_bbox)                     # 找出真實box和預測box的ciou
            
            if iou >= (iou_threshold+0.05):   # 如果>=iou，才是有預測到，TP會增加
                y_true.append(1)
                y_score.append(score)
            else:
                y_true.append(0)
                y_score.append(score)
        elif true_bbox != 'NO' and pred_bbox == 'NO':   # FN
            y_true.append(1)
            y_score.append(0)
        elif true_bbox == 'NO' and pred_bbox != 'NO':   # FP
            
            y_true.append(0)
            y_score.append(score)

    y_trues.append(y_true)
    y_scores.append(y_score)

plt_precision_recall_curve_2(iou_thresholds, y_trues, y_scores)     # 畫PR curve